In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [16]:
q = """
select 
    *
from 
    custom.arch_final_v_plans
where
    season_name like '%Panthers Hockey%'
    and plan_event_name like '%FSF%'
    or plan_event_name like '%FSA%'
    or plan_event_name like '%FSB%'
    or plan_event_name like '%FSP%'
    or plan_event_name like '%FSC%'"""

initial_df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

initial_df['seat_nunm']

KeyError: 'seat'

In [11]:
# create season

initial_df['season'] = [text.split(' ')[0] for text in initial_df['season_name']]

In [15]:
def explode_block_seating(df: pd.DataFrame) -> pd.DataFrame:

    """
        Required Fields
            - seat_num  - last_seat

            - block_purchase_price
            - paid amount
    """

    ## num seats
    # df['seat_list'] = df.apply(lambda row: list(range(row['seat_num'], row['last_seat'] + 1)), axis = 1)
    df['seat_list'] = df.apply(lambda row: list(range(row['seat_num'], row['seat_num'] + row['num_seats'])), axis = 1)

    ## break out monies
    if "block_purchase_price" in df.columns:
        df['gross_rev_seat'] = df.apply(lambda row: row['block_purchase_price'] / len(row['seat_list']), axis = 1)

    if "paid_amount" in df.columns:
        df['paid_amt_seat'] = df.apply(lambda row: (row['paid_amount'] / row['total_events']) / len(row['seat_list']), axis = 1)

    if "original_block_purchase_price" in df.columns:
        df['orig_bpp_seat'] = df.apply(lambda row: row['original_block_purchase_price'] / len(row['seat_list']), axis = 1)

    ## explode
    df = df.explode('seat_list')

    ## rename
    df = df.drop(['seat_num', 'last_seat'], axis = 1).rename(
        columns = {'seat_list' : 'seat'}
        ).reset_index(drop = True)

    if "block_purchase_price" in df.columns: 
        df = df.drop(['block_purchase_price'], axis = 1)
        df = df.rename(columns = {'gross_rev_seat' : 'block_purchase_price'})

    if "paid_amount" in df.columns: 
        df = df.drop(['paid_amount'], axis = 1)
        df = df.rename(columns = {'paid_amt_seat' : 'paid_amount'})
    
    if "original_block_purchase_price" in df.columns: 
        df = df.drop(['original_block_purchase_price'], axis = 1)
        df = df.rename(columns = {'orig_bpp_seat' : 'original_block_purchase_price'})
    
    return df

initial_df = explode_block_seating(initial_df)

TypeError: 'float' object cannot be interpreted as an integer

In [5]:
# Create pc fields

def break_out_price_code(df: pd.DataFrame) -> pd.DataFrame:

    df['pc_one'] = df['price_code'].str[0]
    df['pc_two'] = df['price_code'].str[1]
    df['pc_three'] = df['price_code'].str[2]
    df['pc_four'] = df['price_code'].str[3]

    df['pc_one_two'] = df['price_code'].str[:2]
    df['pc_two_three'] = df['price_code'].str[1:3]
    df['pc_three_four'] = df['price_code'].str[2:4]
    df['pc_two_three_four'] = df['price_code'].str[1:4]

    my_cols = ['pc_one', 'pc_two', 'pc_three', 'pc_four', 'pc_one_two', 'pc_two_three', 'pc_three_four', 'pc_two_three_four']

    for col in my_cols:
        df[col] = df[col].fillna("")

    ## Add "I" to pc_two if price code is a single character
    ## Check for asterisks!
    
    #df['pc_two'] = df.apply(lambda row: "I" if len(row['price_code']) == 1 or row['pc_two'] == "*" else row['pc_two'], axis = 1)

    return df 

initial_df = break_out_price_code(initial_df)

# Create location_general

def create_location_general(df: pd.DataFrame) -> pd.DataFrame:

    my_loc_dict = {
        "General" : [
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
            '1', '2', '3', '4', '5', '6', '7', '8'
            ],
        "Suites" : ['U', 'V', 'W'],
        "Amerant Vault" : ['X'],
        "Loft" : ['Y'],
        "Corona" : ['Z'],
        "Parking" : ['9']
    }

    df['location_general'] = "Other"

    if 'pc_one' in df.columns:
        for key, value in my_loc_dict.items():
            df.loc[df['pc_one'].isin(value), 'location_general'] = key
    else:
        print("pc_one does not exist!")

    return df

initial_df = create_location_general(initial_df)

# create location specific

def create_location_specific(df: pd.DataFrame) -> pd.DataFrame:
    
    my_loc_dict = {
        "Lowers" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8'],
        "Clubs" : ['K', 'L', 'M'],
        "Uppers" : ['N', 'O', 'P', 'Q', 'R', 'S', 'T'],
        "Suites" : ['U', 'V', 'W'],
        "Lounge 954" : ['X'],
        "Loft" : ['Y'],
        "Corona" : ['Z']
    }

    df['location_specific'] = "Other"

    if 'pc_one' in df.columns:
        for key, value in my_loc_dict.items():
            df.loc[df['pc_one'].isin(value), 'location_specific'] = key
    else:
        print("pc_one does not exist!")

    return df

initial_df = create_location_specific(initial_df)

# create arena level

def create_arena_level(df: pd.DataFrame) -> pd.DataFrame:

    my_level_dict = {
        "Lowers" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', '1', '2', '3', '4', '5', '6', '7', '8'],
        "Clubs" : ['X', 'Z'],
        "Uppers" : ['N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y'],
        "Suites" : ['U', 'V', 'W'],
    }

    df['arena_level'] = "Other"

    if 'pc_one' in df.columns:
        for key, value in my_level_dict.items():
            df.loc[(df['pc_one'].isin(value)), 'arena_level'] = key
    else:
        print("pc_one does not exist!")

    return df

initial_df = create_arena_level(initial_df)

# create is premium

def create_is_premium(df: pd.DataFrame) -> pd.DataFrame:

    df['is_premium'] = False
    df.loc[(df['pc_one'].isin(['U', 'V', 'W', 'X', 'Z', '1', '2'])), 'is_premium'] = True

    df['is_premium_text'] = ["Premium" if x is True else "General" for x in df['is_premium']]

    return df

initial_df = create_is_premium(initial_df)

In [6]:
# Create plan_type

def create_plan_type(df: pd.DataFrame) -> pd.DataFrame:
        
    '''
        this_df['plan_type'] = "Unknown"
        this_df.loc[(this_df['pc_two'] == "F"), 'plan_type'] = "Full"
        this_df.loc[(this_df['pc_two'] == "C"), 'plan_type'] = "Premier"
        this_df.loc[(this_df['pc_two'].isin(["A", "B", "H"])), 'plan_type'] = "Half"
        this_df.loc[(this_df['pc_two'].isin(["A", "B", "H"])) & ((this_df['pc_three'] == "F") | (this_df['pc_four'] == "F")), 'plan_type'] = "Full"
        this_df.loc[(this_df['pc_two'].isin(["I", "*"])), 'plan_type'] = "Singles"
        this_df.loc[(this_df['pc_two_three'] == "IX"), 'plan_type'] = "Flex"
        this_df.loc[(this_df['pc_two'] == "G"), 'plan_type'] = "Groups"
        this_df.loc[(this_df['pc_two'] == "C") & (this_df['pc_three'].isin(["I", "E"])), 'plan_type'] = "Singles"
        this_df.loc[(this_df['pc_one'].isin(["U", "V", "W"])) & (this_df['pc_two'].isin(["Y", "R", "E", "N"])), 'plan_type'] = "Full"
    '''

    df['plan_type'] = "Unknown"
    df.loc[(df['pc_two'] == "F"), 'plan_type'] = "Full"
    df.loc[(df['pc_two'] == "P"), 'plan_type'] = "Premier"
    df.loc[(df['pc_two'].isin(["I", "*", ""])), 'plan_type'] = "Singles"
    df.loc[(df['pc_two'] == "X"), 'plan_type'] = "Flex"
    df.loc[(df['pc_two'] == "G"), 'plan_type'] = "Groups"

        ## SAVED ##
        # my_dict = {
        #     "22FSF" : "Full",
        #     "22FSP" : "Premier",
        # }

        # df['plan_type'] = "Unknown"

        # for key, value in my_dict.items():
        #     df.loc[df['event_name'] == key, 'plan_type'] = value

    return df

initial_df = create_plan_type(initial_df)

# create is logitix

def create_is_logitix(df: pd.DataFrame) -> pd.DataFrame:

    df['is_logitix'] = False
    df.loc[(df['pc_two_three'].isin(["FD", "GD"])) & (~df['pc_one'].isin(["U", "V", "W"])), 'is_logitix'] = True 

    return df

initial_df = create_is_logitix(initial_df)

# create is sponsor

def create_is_sponsor(df: pd.DataFrame) -> pd.DataFrame:

    # df['is_sponsor'] = df.apply(lambda row: True if row['acct_type_desc'] == "Sponsor" else False, axis = 1)
        
    #df['is_sponsor'] = df.apply(lambda row: True if row['rep_full_name'] == "Sponsor" else False, axis = 1)

    df['is_sponsor'] = [True if x == "Sponsor" else False for x in df['acct_rep_full_name']]

    return df

initial_df = create_is_sponsor(initial_df)

# create is trade

def create_is_trade(df: pd.DataFrame) -> pd.DataFrame:

    # trade_vals = ["Trade", "Marketing Trade", "Media Trade", "Sponsor Trade", "Trade-Marketing", "Trade-Sponsor"]
    # df['is_trade'] = df.apply(lambda row: True if row['acct_type_desc'] in trade_vals else False, axis = 1)

    #df['is_trade'] = df.apply(lambda row: True if row['rep_full_name'].find("Trade") > -1 else False, axis = 1)

    df['is_trade'] = [True if str(x).find("Trade") > -1 else False for x in df['acct_rep_full_name']]

    return df

initial_df = create_is_trade(initial_df)

# create account type

def create_account_type(df: pd.DataFrame) -> pd.DataFrame:

    df['account_type'] = "Standard"

    df.loc[df['is_logitix'] == True, 'account_type'] = "Broker"
    df.loc[df['is_sponsor'] == True, 'account_type'] = "Sponsor"
    df.loc[df['is_trade'] == True, 'account_type'] = "Sponsor"

    return df

initial_df = create_account_type(initial_df)

# create account type description group

def create_acct_type_desc_group(df: pd.DataFrame) -> pd.DataFrame:

    corporate_values = ["Sponsor", "Business", "Broker", "Community", "Suite", "Marketing", "PR", "Trade", "Box Office"]

    if 'acct_type_desc' in df.columns:

        df['acct_type_desc_group'] = "Personal"
        
        for val in corporate_values:
            df.loc[(df['acct_type_desc'].str.find(val)) >= 0, 'acct_type_desc_group'] = "Corporate"
    else:
        print("acct_type_desc does not exist!")

    return df

initial_df = create_acct_type_desc_group(initial_df)

acct_type_desc does not exist!


In [7]:
# Create ticket_type

def create_ticket_type(df: pd.DataFrame) -> pd.DataFrame:

    df['ticket_type'] = "Unknown"

    df.loc[
        (df['plan_type'] == "Full") & (df['account_type'] == "Standard") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Full"

    df.loc[
        (df['plan_type'] == "Flex") & (df['account_type'] == "Standard") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Flex"

    df.loc[
        (df['plan_type'] == "Premier") & (df['account_type'] == "Standard") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Premier"

    df.loc[
        (df['plan_type'].isin(["Full", "Premier"])) & (df['location_general'] == "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Annual Suites"

    df.loc[
        (df['plan_type'].isin(["Full", "Flex", "Premier"])) & (df['account_type'] == "Sponsor") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Sponsor"

    df.loc[(df['is_trade'] == True), 'ticket_type'] = "Trade"

        ## Double check with Ty
        # suite_price_codes_full = ["VFRW", "VFRX", "WFRW", "WFRX", "VFRY", "VFRZ", "WFRY", "WFRZ", "VFN", "VFNF", "WFN", "WFNF", "WFNX"]
        # suite_price_codes_half = ["VAN1", "WAN1", "VBN1", "WBN1"]

        # df.loc[df['price_code'].isin(suite_price_codes_full), 'ticket_type'] = "Full"
        # df.loc[df['price_code'].isin(suite_price_codes_half), 'ticket_type'] = "Half"
        #######################

    df.loc[
        (df['plan_type'] == "Groups") & (df['account_type'] != "Broker") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Groups"

    df.loc[
        (df['plan_type'] == "Singles") & (df['account_type'] != "Broker") & (df['location_general'] != "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Singles"

    df.loc[
        (df['plan_type'].isin(["Singles", "Groups", "Flex"])) & (df['account_type'] != "Broker") & (df['location_general'] == "Suites") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Nightly Suites"
            
    df.loc[
        (df['plan_type'].isin(["Singles", "Groups", "Flex"])) & (df['account_type'] != "Broker") & (df['pc_one_two'] == "YI") & (df['is_trade'] == False)
        , 'ticket_type'
        ] = "Nightly Suites"

    df.loc[
        (df['is_trade'] == False) & (df['account_type'] == "Broker")
        , 'ticket_type'
        ] = "Secondary"

    return df

initial_df = create_ticket_type(initial_df)

In [8]:
# create is_renewal

def create_is_renewal(df: pd.DataFrame) -> pd.DataFrame:

    df['is_renewal'] = False
    # df.loc[((df['pc_three'] == "R") & (df['pc_two_three'] != "GR")) & (~df['pc_one'].isin(["U", "V", "W"])), 'is_renewal'] = True
    df.loc[((df['pc_three'] == "R") & (df['pc_two_three'] != "GR")), 'is_renewal'] = True

    df['is_renewal_text'] = ["Renewal" if x is True else "New Business" for x in df['is_renewal']]

    return df

initial_df = create_is_renewal(initial_df)

# create is upgrade

def create_is_upgrade(df: pd.DataFrame) -> pd.DataFrame:

    df['is_upgrade'] = False
    df.loc[((df['pc_three_four'] == "RU") & (df['location_general'] != "Suites")), 'is_upgrade'] = True

    df['is_renewal_and_upgrade_text'] = df['is_renewal_text'] 
    df.loc[(df['is_upgrade'] == True), 'is_renewal_and_upgrade_text'] = "Upgrade"

    return df

initial_df = create_is_upgrade(initial_df)

In [50]:
#create comp seats

def create_is_comp(df: pd.DataFrame) -> pd.DataFrame:
        
    #df['is_comp'] = df.apply(lambda row: True if row['block_purchase_price'] == 0 else False, axis = 1)

    df['is_comp'] = [True if x == 0 else False for x in df['block_purchase_price']]

    return df

initial_df = create_is_comp(initial_df)

# create comp/paid seats

def create_paid_comp_seats(df: pd.DataFrame) -> pd.DataFrame:

    """
        - seats must be exploded
    """

    #df['paid_seats'] = df.apply(lambda row: 1 if row['is_comp'] is False else 0, axis = 1)
    df['paid_seats'] = [1 if x is False else 0 for x in df['is_comp']]
    #df['comp_seats'] = df.apply(lambda row: 1 if row['is_comp'] is True else 0, axis = 1)
    df['comp_seats'] = [1 if x is True else 0 for x in df['is_comp']]

    return df

initial_df = create_paid_comp_seats(initial_df)

# create total seats

initial_df['total_seats'] = initial_df['paid_seats']+initial_df['comp_seats']

In [9]:
# create gross/net revenue

def create_food_fee(df: pd.DataFrame) -> pd.DataFrame:

    '''
        - seats must be exploded
    '''

    df['food_fee'] = 0

    df.loc[df['pc_one'] == "Z", 'food_fee'] = 25
    df.loc[df['pc_one'] == "Y", 'food_fee'] = 15
    df.loc[df['pc_one'] == "X", 'food_fee'] = 35

    df['food_fee'] = df.apply(
        lambda row: row['food_fee'] if row['is_comp'] is False else 0, axis = 1
    )

    return df

initial_df = create_food_fee(initial_df)

def create_gross_without_food(df: pd.DataFrame) -> pd.DataFrame:

    if 'event_food_fee' in df.columns:
        df['gross_without_food'] = df['block_purchase_price'] - df['event_food_fee']
    else:
        df['gross_without_food'] = df['block_purchase_price'] - df['food_fee']

    return df

initial_df = create_gross_without_food(initial_df)

def create_net_revenue_internal(df: pd.DataFrame) -> pd.DataFrame:

    '''
         - needs seats exploded
    '''

    #df['net_revenue'] = 0
    #df.loc[(df['is_host'] == True), 'net_revenue'] = df['gross_without_food'] / 1.07
    #df.loc[(df['is_host'] == False), 'net_revenue'] = (df['pc_ticket'] + df['pc_licfee'] + df['surchg_amount']) / 1.07

    # df['net_revenue'] = df.apply(
    #     lambda row: (row['gross_without_food'] / 1.07) if rowc['is_host'] == True
    #         else ((row['pc_ticket'] + row['pc_licfee'] + row['surchg_amount']) / 1.07), axis = 1
    # )

    df['net_revenue'] = df['gross_without_food'] / 1.07

    return df

initial_df = create_net_revenue_internal(initial_df) 

KeyError: 'is_comp'

In [ ]:
### Season - DONE
### Is Logitix - DONE
### Is Sponsor - DONE
### Is Trade - DONE
### Account Type - DONE
### Account Type Description Group - DONE
### Ticket Type - DONE
### Is Renewal - DONE
### Is Comp - DONE
### Paid Comp Seats - DONE
### Paid Seats - DONE
### Comp Seats - DONE
### Total Seats - DONE
### Gross Revenue - DONE
### Net Revenue - DONE
### Location General - DONE
### Location Specific - DONE

In [53]:
# FLA_Redshift(**rs_creds).write_to_warehouse(df = final_df, table_name= "cth_v_plans_update")

c:\Users\riffere\Documents\pandas_party\venv\Lib\site-packages\catnip\fla_redshift.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_date'] = datetime.utcnow()


In [ ]:
def transform(df: pd.DataFrame) -> pd.DataFrame:

    start_time = datetime.now()

    df['last_seat'] = [x + (y-1) for (x,y) in zip(df['seat_num'], df['num_seats'])]

    df = break_out_price_code(df)
    df = explode_block_seating(df)

    df['paid_amount'] = df['block_purchase_price'] * df['percent_paid']
    df['owed_amount'] = df['block_purchase_price'] * (1 - df['percent_paid'])

    df = Account_Fields().create_is_logitix_2223(df); print("5"); print(datetime.now() - start_time)
    df = Account_Fields().create_is_sponsor(df); print("6"); print(datetime.now() - start_time)
    df = Account_Fields().create_is_trade(df); print("7"); print(datetime.now() - start_time)
    df = Account_Fields().create_account_type(df); print("8"); print(datetime.now() - start_time)
    df = Account_Fields().create_acct_type_desc_group(df); print("9"); print(datetime.now() - start_time)
    
    df = Financial_Fields().create_is_comp(df); print("10"); print(datetime.now() - start_time)
    df = Financial_Fields().create_paid_comp_seats(df); print("11"); print(datetime.now() - start_time)

    df = Location_Fields().create_location_general(df); print("12"); print(datetime.now() - start_time)
    df = Location_Fields().create_location_specific(df); print("13"); print(datetime.now() - start_time)
    df = Location_Fields().create_arena_level(df); print("14"); print(datetime.now() - start_time)
    df = Location_Fields().create_is_premium(df); print("15"); print(datetime.now() - start_time)

    df = Ticket_Type_Fields().create_is_renewal_2223(df); print("16"); print(datetime.now() - start_time)
    df = Ticket_Type_Fields().create_is_upgrade_2223(df); print("17"); print(datetime.now() - start_time)
    df = Ticket_Type_Fields().create_plan_type_2223(df); print("18"); print(datetime.now() - start_time)
    df = Ticket_Type_Fields().create_ticket_type_2223(df); print("19"); print(datetime.now() - start_time)

    df['fse_calc'] = 0
    df.loc[(df['plan_event_name'].str.contains('FS')) & (df['is_comp'] == False), 'fse_calc'] = 1/42 * df['total_events']

    ## seat ids
    df['seat_id'] = [f"{x}-{y}-{z}" for (x,y,z) in zip(df['section_name'], df['row_name'], df['seat'])]
    df['seat_event_id'] = [f"{_a}-{x}-{y}-{z}" for (x,y,z,_a) in zip(df['section_name'], df['row_name'], df['seat'], df['plan_event_name'])]


    df = df[[c for c in df if c != 'processed_date'] + ['processed_date']]

    return df 

In [ ]:
class Ticket_Type_Fields():

    @staticmethod
    def create_is_upgrade_2223(df: pd.DataFrame) -> pd.DataFrame:

        df['is_upgrade'] = False
        df.loc[((df['pc_three_four'] == "RU") & (df['location_general'] != "Suites")), 'is_upgrade'] = True

        df['is_renewal_and_upgrade_text'] = df['is_renewal_text'] 
        df.loc[(df['is_upgrade'] == True), 'is_renewal_and_upgrade_text'] = "Upgrade"

        return df

    @staticmethod
    def create_is_renewal_2223(df: pd.DataFrame) -> pd.DataFrame:

        '''
            - check with Ty on parameters
        '''

        df['is_renewal'] = False
        df.loc[((df['pc_three'] == "R") & (df['pc_two_three'] != "GR")), 'is_renewal'] = True
        # df.loc[(df['pc_two'] == "F") & (df['pc_three'].isin([str(i) for i in range(0, 10)])), 'is_renewal'] = True
        # df.loc[(df['pc_two_three'] == "HF"), 'is_renewal'] = True
        # df.loc[(df['pc_one'].isin(["U", "V", "W"])) & (df['pc_two'] == "R"), 'is_renewal'] = True

        # df['is_renewal_text'] = df.apply(lambda row: "Renewal" if row['is_renewal'] is True else "New Business", axis = 1)
        df['is_renewal_text'] = ["Renewal" if x is True else "New Business" for x in df['is_renewal']]

        return df
    
    @staticmethod
    def create_is_host(df: pd.DataFrame) -> pd.DataFrame:

        #df['is_host'] = df.apply(lambda row: True if len(row['price_code']) == 1 or row['pc_two'] == '*' else False, axis = 1)

        df['is_host'] = [True if x == '*' else False for x in df['pc_two']]

        return df

    @staticmethod
    def create_is_all_inclusive_2223(df: pd.DataFrame) -> pd.DataFrame:

        df['is_all_inclusive'] = df.apply(
            lambda row: True if (str(row['price_code_desc']).lower().find("inclusive") >= 0) or (str(row['price_code_group']) == "AI") else False, axis = 1
        )

        return df

    @staticmethod
    def create_ticket_type_playoffs_2223(df: pd.DataFrame) -> pd.DataFrame:

        df['ticket_type_playoffs'] = "Unknown"

        df.loc[
            (df['plan_type'] == "Full") & (df['is_renewal'] == True) & (df['pc_four'] != "A")
            , 'ticket_type_playoffs'
            ] = "Renewal Strips"

        df.loc[
            (df['plan_type'] == "Full") & (df['is_renewal'] == True) & (df['pc_four'] == "A")
            , 'ticket_type_playoffs'
            ] = "Add-on Strips"

        df.loc[
            (df['plan_type'] == "Full") & (df['is_renewal'] == False)
            , 'ticket_type_playoffs'
            ] = "New Business Strips"

        df.loc[
            (df['account_type'] == "Broker")
            , 'ticket_type_playoffs'
            ] = "Secondary"
        
        df.loc[
            (df['pc_two'].isin(["*", "I"]))
            , 'ticket_type_playoffs'
            ] = "Singles"

        return df
    
    @staticmethod
    def create_hrr_plan_type_2223(df: pd.DataFrame) -> pd.DataFrame:

        if "plan_type" not in df.columns:
            df = Ticket_Type_Fields().create_plan_type_2223(df)

        df['hrr_plan_type'] = "Plans"
        df.loc[(df['is_all_inclusive'] == True), 'hrr_plan_type'] = "All-Inclusive"
        df.loc[(df['plan_type'] == "Groups"), 'hrr_plan_type'] = "Groups"
        df.loc[(df['plan_type'] == "Singles"), 'hrr_plan_type'] = "Singles"

        return df

    @staticmethod
    def create_fse(df: pd.DataFrame) -> pd.DataFrame:

        df['fse_calc'] = 0
        df.loc[(~df['ticket_type'].isin(['Singles', 'Nightly Suites', 'Groups', 'Flex'])) & (df['pc_two_three'] != "GD") & (df['is_comp'] == False), 'fse_calc'] = 1/42

        return df 

    @staticmethod
    def create_fse_2324(df: pd.DataFrame) -> pd.DataFrame:

        # df['fse_calc'] = 0
        # df.loc[(df['ticket_type'].isin(["Full", "Annual Suites"])) & (df['is_comp'] == False), 'fse_calc'] = 1
        # df.loc[(df['ticket_type'] == "Premier") & (df['is_comp'] == False), 'fse_calc'] = 25/42

        df['fse_calc'] = 0
        df.loc[(df['plan_event_name'] == "23FSF") & (df['is_comp'] == False), 'fse_calc'] = 1
        df.loc[(df['plan_event_name'] == "23FSP") & (df['is_comp'] == False), 'fse_calc'] = 25/42

        return df 


In [ ]:
class Location_Fields():

    @staticmethod
    def create_is_sro(df: pd.DataFrame) -> pd.DataFrame:

        matches = ["GA", "SRO"]

        if 'row_name' in df.columns:
            df['is_sro'] = df.apply(lambda row: True if any(x in row['row_name'] for x in matches) else False, axis = 1)

        return df


    @staticmethod
    def create_location_specific(df: pd.DataFrame) -> pd.DataFrame:
        
        my_loc_dict = {
            "Lowers" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8'],
            "Clubs" : ['K', 'L', 'M'],
            "Uppers" : ['N', 'O', 'P', 'Q', 'R', 'S', 'T'],
            "Suites" : ['U', 'V', 'W'],
            "Lounge 954" : ['X'],
            "Loft" : ['Y'],
            "Corona" : ['Z']
        }

        df['location_specific'] = "Other"

        if 'pc_one' in df.columns:
            for key, value in my_loc_dict.items():
                df.loc[df['pc_one'].isin(value), 'location_specific'] = key
        else:
            print("pc_one does not exist!")

        return df


    @staticmethod
    def create_location_events(df: pd.DataFrame) -> pd.DataFrame:

        my_loc_dict = {
            "ENN's" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T'],
            "Loge Box" : ['U'],
            "Club Suites" : ['V'],
            "Suites" : ['W'],
            "Lounge 954" : ['X'],
            "Loft" : ['Y'],
            "Corona" : ['Z']
        }

        df['location_events'] = "Other"

        if 'pc_one' in df.columns:
            for key, value in my_loc_dict.items():
                df.loc[df['pc_one'].isin(value), 'location_events'] = key
        else:
            print("pc_one does not exist!")

        return df


    @staticmethod
    def create_location_events_general(df: pd.DataFrame) -> pd.DataFrame:

        my_loc_dict = {
            "ENN's" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T'],
            "Suites": ['U', 'V', 'W'],
            "Clubs": ['X', 'Y', 'Z']
        }

        df['location_events_general'] = "Other"

        if 'pc_one' in df.columns:
            for key, value in my_loc_dict.items():
                df.loc[df['pc_one'].isin(value), 'location_events_general'] = key
        else:
            print("pc_one does not exist!")

        return df


    @staticmethod
    def create_location_hrr(df: pd.DataFrame) -> pd.DataFrame:

        if "location_general" not in df.columns:
            df = Location_Fields().create_location_general(df)
        
        df['location_general'] = df['location_general'].replace("General", "Bowl")
        df = df.rename(columns = {'location_general' : 'hrr_location'})

        return df


    @staticmethod
    def create_arena_level(df: pd.DataFrame) -> pd.DataFrame:

        my_level_dict = {
            "Lowers" : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', '1', '2', '3', '4', '5', '6', '7', '8'],
            "Clubs" : ['X', 'Z'],
            "Uppers" : ['N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y'],
            "Suites" : ['U', 'V', 'W'],
        }

        df['arena_level'] = "Other"

        if 'pc_one' in df.columns:
            for key, value in my_level_dict.items():
                df.loc[(df['pc_one'].isin(value)), 'arena_level'] = key
        else:
            print("pc_one does not exist!")

        return df

    @staticmethod
    def create_arena_level_internal(df: pd.DataFrame) -> pd.DataFrame:

        my_level_dict = {
            "Lowers": ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', '1', '2', '3', '4', '5', '6', '7', '8'],
            "Clubs": ['K', 'L', 'M'],
            "Uppers": ['N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Y'],
            "Suites": ['U', 'V', 'W'],
            "Premium": ['X', 'Z']
        }

        df['arena_level_internal'] = "Other"

        if 'pc_one' in df.columns:
            for key, value in my_level_dict.items():
                df.loc[(df['pc_one'].isin(value)), 'arena_level_internal'] = key
        else:
            print("pc_one does not exist!")

        return df

    @staticmethod
    def add_xy_coordinates(df: pd.DataFrame) -> pd.DataFrame:

        seat_loc_df = FLA_Redshift().query_warehouse("SELECT * FROM custom.cth_xy_coordinates")

        if 'seat_id' in df.columns:
            df = pd.merge(seat_loc_df, df, how = "left", on = "seat_id")
        else:
            print("seat_id does not exist")

        return df

    @staticmethod
    def create_is_premium(df: pd.DataFrame) -> pd.DataFrame:

        df['is_premium'] = False
        df.loc[(df['pc_one'].isin(['U', 'V', 'W', 'X', 'Z', '1', '2'])), 'is_premium'] = True

        df['is_premium_text'] = ["Premium" if x is True else "General" for x in df['is_premium']]

        return df


In [ ]:
class Account_Fields():

    @staticmethod
    def create_is_logitix_playoffs_2223(df: pd.DataFrame) -> pd.DataFrame:

        df['is_logitix'] = False
        df.loc[(df['pc_four'] == "L"), 'is_logitix'] = True 
        df.loc[(df['pc_two_three'] == "GL"), 'is_logitix'] = True 

        return df

    @staticmethod
    def create_hrr_account_type(df: pd.DataFrame) -> pd.DataFrame:

        df['hrr_account_type'] = "Standard"
        df.loc[df['acct_type_desc'] == "Sponsor", 'hrr_account_type'] = "Sponsor"
        df.loc[df['acct_type_desc'].isin(["Sponsor Trade", "Trade-Sponsor"]), 'hrr_account_type'] = "Sponsor Trade"
        df.loc[df['acct_type_desc'].isin(["Marketing Trade", "Trade-Marketing"]), 'hrr_account_type'] = "Marketing Trade"
        df.loc[df['acct_type_desc'] == "Trade", 'hrr_account_type'] = "Trade"
        
        return df

    @staticmethod
    def create_acct_type_desc_group(df: pd.DataFrame) -> pd.DataFrame:

        corporate_values = ["Sponsor", "Business", "Broker", "Community", "Suite", "Marketing", "PR", "Trade", "Box Office"]

        if 'acct_type_desc' in df.columns:

            df['acct_type_desc_group'] = "Personal"
            
            for val in corporate_values:
                df.loc[(df['acct_type_desc'].str.find(val)) >= 0, 'acct_type_desc_group'] = "Corporate"
        else:
            print("acct_type_desc does not exist!")

        return df 


def explode_block_seating(df: pd.DataFrame) -> pd.DataFrame:

    """
        Required Fields
            - seat_num  - last_seat

            - block_purchase_price
            - paid amount
    """

    ## num seats
    df['seat_list'] = df.apply(lambda row: list(range(row['seat_num'], row['last_seat'] + 1)), axis = 1)

    ## break out monies
    if "block_purchase_price" in df.columns:
        df['gross_rev_seat'] = df.apply(lambda row: row['block_purchase_price'] / len(row['seat_list']), axis = 1)

    if "paid_amount" in df.columns:
        df['paid_amt_seat'] = df.apply(lambda row: (row['paid_amount'] / row['total_events']) / len(row['seat_list']), axis = 1)

    if "original_block_purchase_price" in df.columns:
        df['orig_bpp_seat'] = df.apply(lambda row: row['original_block_purchase_price'] / len(row['seat_list']), axis = 1)

    ## explode
    df = df.explode('seat_list')

    ## rename
    df = df.drop(['seat_num', 'last_seat'], axis = 1).rename(
        columns = {'seat_list' : 'seat'}
        ).reset_index(drop = True)

    if "block_purchase_price" in df.columns: 
        df = df.drop(['block_purchase_price'], axis = 1)
        df = df.rename(columns = {'gross_rev_seat' : 'block_purchase_price'})

    if "paid_amount" in df.columns: 
        df = df.drop(['paid_amount'], axis = 1)
        df = df.rename(columns = {'paid_amt_seat' : 'paid_amount'})
    
    if "original_block_purchase_price" in df.columns: 
        df = df.drop(['original_block_purchase_price'], axis = 1)
        df = df.rename(columns = {'orig_bpp_seat' : 'original_block_purchase_price'})
    
    return df


def create_seat_ids(df: pd.DataFrame) -> pd.DataFrame:

    #df['seat_id'] = df.apply(lambda row: f"{row['section_name']}-{row['row_name']}-{row['seat']}", axis = 1)
    df['seat_id'] = [f"{x}-{y}-{z}" for (x,y,z) in zip(df['section_name'], df['row_name'], df['seat'])]

    #df['seat_event_id'] = df.apply(lambda row: f"{str(row['event_name'])}-{row['seat_id']}", axis = 1)
    df['seat_event_id'] = [f"{_a}-{x}-{y}-{z}" for (x,y,z,_a) in zip(df['section_name'], df['row_name'], df['seat'], df['event_name'])]


    return df


def gameday_check() -> bool:

    q = """
        SELECT 
            SUM( 
                CASE 
                    WHEN event_date = CAST(GETDATE() AS DATE) THEN 1
                    ELSE 0
                END 
            ) AS "gameday_check"
        FROM 
            custom.cth_game_descriptions cgd 
    """

    return bool(FLA_Redshift().query_warehouse(sql_string = q).iloc[0,0])


def create_is_game_over(df: pd.DataFrame) -> pd.DataFrame:

    # df['is_game_over'] = df.apply(
    #     lambda row: True if datetime.strptime(row['event_date'], "%Y-%m-%d").date() < date.today() else False, axis = 1
    # )

    df['is_game_over'] = [True if datetime.strptime(x, "%Y-%m-%d").date() < (datetime.now() - timedelta(hours = 1)).date() else False for x in df['event_date']]

    return df 


def create_tier_from_event_name(df: pd.DataFrame) -> pd.DataFrame:

    df['tier'] = df['event_name'].str[-1]

    return df


def create_days_out_from_event(df: pd.DataFrame) -> pd.DataFrame:

    df['days_out_from_event'] = [FLA_Helpers().get_days_between(datetime.strftime(x, '%Y-%m-%d'), y) for (x,y) in zip(df['add_datetime'], df['event_date'])]

    return df 

def create_days_out_bucket(df: pd.DataFrame) -> pd.DataFrame:

    df['days_out_bucket'] = "Unknown"
    df.loc[df['days_out_from_event'] <= 0, 'days_out_bucket'] = "A: Day Of"
    df.loc[df['days_out_from_event'].isin([*range(1, 4, 1)]), 'days_out_bucket'] = "B: 1-3 Days Out"
    df.loc[df['days_out_from_event'].isin([*range(4, 8, 1)]), 'days_out_bucket'] = "C: 4-7 Days Out"
    df.loc[df['days_out_from_event'].isin([*range(8, 15, 1)]), 'days_out_bucket'] = "D: 1-2 Weeks Out"
    df.loc[df['days_out_from_event'].isin([*range(15, 31, 1)]), 'days_out_bucket'] = "E: 2-4 Weeks Out"
    df.loc[df['days_out_from_event'].isin([*range(31, 61, 1)]), 'days_out_bucket'] = "F: 1-2 Months Out"
    df.loc[df['days_out_from_event'].isin([*range(61, 91, 1)]), 'days_out_bucket'] = "G: 2-3 Months Out"
    df.loc[df['days_out_from_event'] >= 91, 'days_out_bucket'] = "H: 3+ Months Out"

    return df 

In [ ]:
class Financial_Fields():

    @staticmethod
    def create_is_comp(df: pd.DataFrame) -> pd.DataFrame:
        
        #df['is_comp'] = df.apply(lambda row: True if row['block_purchase_price'] == 0 else False, axis = 1)

        df['is_comp'] = [True if x == 0 else False for x in df['block_purchase_price']]

        return df

    @staticmethod
    def create_paid_comp_seats(df: pd.DataFrame) -> pd.DataFrame:

        """
            - seats must be exploded
        """

        #df['paid_seats'] = df.apply(lambda row: 1 if row['is_comp'] is False else 0, axis = 1)
        df['paid_seats'] = [1 if x is False else 0 for x in df['is_comp']]
        #df['comp_seats'] = df.apply(lambda row: 1 if row['is_comp'] is True else 0, axis = 1)
        df['comp_seats'] = [1 if x is True else 0 for x in df['is_comp']]

        return df

    @staticmethod
    def create_food_fee_2223(df: pd.DataFrame) -> pd.DataFrame:

        '''
            - seats must be exploded
        '''

        df['food_fee'] = 0

        df.loc[df['pc_one'] == "Z", 'food_fee'] = 25
        df.loc[df['pc_one'] == "Y", 'food_fee'] = 15
        df.loc[df['pc_one'] == "X", 'food_fee'] = 35

        df['food_fee'] = df.apply(
            lambda row: row['food_fee'] if row['is_comp'] is False else 0, axis = 1
        )

        return df

    @staticmethod
    def create_food_fee_2223_events(df: pd.DataFrame) -> pd.DataFrame:

        ## - seats must be exploded

        df['food_fee'] = 0

        df.loc[(df['pc_one'] == "Z") & (df['pc_three'] != "N"), 'food_fee'] = 25
        df.loc[(df['pc_one'] == "X") & (df['pc_three'] != "N"), 'food_fee'] = 35

        df.loc[df['is_comp'] == True, 'food_fee'] = 0

        return df

    @staticmethod
    def create_gross_without_food(df: pd.DataFrame) -> pd.DataFrame:

        if 'event_food_fee' in df.columns:
            df['gross_without_food'] = df['block_purchase_price'] - df['event_food_fee']
        else:
            df['gross_without_food'] = df['block_purchase_price'] - df['food_fee']

        return df

    @staticmethod
    def create_net_revenue_internal_2223(df: pd.DataFrame) -> pd.DataFrame:

        '''
            - needs seats exploded
        '''

        df['net_revenue'] = 0
        df.loc[(df['is_host'] == True), 'net_revenue'] = df['gross_without_food'] / 1.07
        df.loc[(df['is_host'] == False), 'net_revenue'] = (df['pc_ticket'] + df['pc_licfee'] + df['surchg_amount']) / 1.07

        # df['net_revenue'] = df.apply(
        #     lambda row: (row['gross_without_food'] / 1.07) if row['is_host'] == True
        #         else ((row['pc_ticket'] + row['pc_licfee'] + row['surchg_amount']) / 1.07), axis = 1
        # )

        return df

    @staticmethod
    def create_net_revenue_internal_2324(df: pd.DataFrame) -> pd.DataFrame:

        '''
            - needs seats exploded
        '''

        df['net_revenue'] = 0
        df.loc[(df['is_host'] == True), 'net_revenue'] = df['gross_without_food'] / 1.07
        df.loc[(df['is_host'] == False), 'net_revenue'] = df['pc_ticket'] + df['pc_licfee'] + df['surchg_amount']

        return df
    
    @staticmethod
    def create_net_revenue_logitix_2223(df: pd.DataFrame) -> pd.DataFrame:

        '''
            - Need to attach pc_one to dynasty sales in order to get food fee
        '''

        # df['net_revenue'] = df.apply(
        #     lambda row: (((row['block_purchase_price'] * 0.93) - row['food_fee']) / 1.07), axis = 1
        # )

        df['net_revenue'] = ((df['block_purchase_price'] * 0.93) - df['food_fee']) / 1.07

        return df

    @staticmethod
    def create_jl_parking_refund_field(df: pd.DataFrame) -> pd.DataFrame:

        df['net_revenue'] = df.apply(
            lambda row: (row['net_revenue'] + (row['num_seats'] * 5)) if row['retail_ticket_type'].isin(['J-TYPE', 'L-TYPE']) else row['net_revenue']
            , axis = 1
        )

        return df

    @staticmethod
    def create_events_commission_rate_2223(df: pd.DataFrame) -> pd.DataFrame:
        
        df['commission_type'] = "Unknown"
        df['commission_rate'] = 0.0

        mask = (df['pc_two'] == "G")
        df.loc[mask, 'commission_type'] = "Single Event Groups"
        df.loc[mask, 'commission_rate'] = 0.04

        mask = ((df['pc_two'] == "G") & (df['rep_department'] == "Events"))
        df.loc[mask, 'commission_type'] = "Single Event Groups"
        df.loc[mask, 'commission_rate'] = 0.08

        mask = (df['location_events_general'].isin(["ENN's", "Clubs"]))
        df.loc[mask, 'commission_type'] = "Single Event Clubs and ENN"
        df.loc[mask, 'commission_rate'] = 0.04

        mask = (df['location_events_general'] == "Suites")
        df.loc[mask, 'commission_type'] = "Single Event Suites"
        df.loc[mask, 'commission_rate'] = 0.08

        return df

    @staticmethod
    def create_hockey_commission_rate_2223(df: pd.DataFrame) -> pd.DataFrame:

        df['commission_type'] = "Unknown"
        df['commission_rate'] = 0.0


        ## New Business ##

        mask = ((df['ticket_type'] == "Full") & (df['is_renewal'] == False) & (df['location_general'] != "Lounge 954") & (df['location_general'] != "Suites"))
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus New"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = ((df['ticket_type'] == "Premier") & (df['is_renewal'] == False) & (df['location_general'] != "Lounge 954") & (df['location_general'] != "Suites"))
        df.loc[mask, 'commission_type'] = "Bowl Premier New"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = ((df['ticket_type'] == "Full") & (df['is_renewal'] == False) & (df['location_general'] == "Lounge 954"))
        df.loc[mask, 'commission_type'] = "Lounge 954 Premier Plus New"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = ((df['ticket_type'] == "Premier") & (df['is_renewal'] == False) & (df['location_general'] == "Lounge 954"))
        df.loc[mask, 'commission_type'] = "Lounge 954 Premier New"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = (
            (df['ticket_type'] == "Full") & (df['is_renewal'] == False) & 
            ((df['location_general'].isin(["Lounge 954", "Corona"])) | (df['pc_one'].isin(["1", "2"]))) &
            (df['rep_department'] == "Premium") & (df['rep_sales_service'] == "Sales")
        )
        df.loc[mask, 'commission_type'] = "Premium Sales Premier Plus New"
        df.loc[mask, 'commission_rate'] = 0.12

        mask = (
            (df['ticket_type'] == "Premier") & (df['is_renewal'] == False) & 
            ((df['location_general'].isin(["Lounge 954", "Corona"])) | (df['pc_one'].isin(["1", "2"]))) &
            (df['rep_department'] == "Premium") & (df['rep_sales_service'] == "Sales")
        )
        df.loc[mask, 'commission_type'] = "Premium Sales Premier New"
        df.loc[mask, 'commission_rate'] = 0.12


        ## Renewals ##

        mask = (
            (df['ticket_type'] == "Full") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_department'] == "Membership") & (df['rep_sales_service'] == "Service") & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus Renewal"
        df.loc[mask, 'commission_rate'] = 0.014

        mask = (
            (df['ticket_type'] == "Premier") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_department'] == "Membership") & (df['rep_sales_service'] == "Service")  & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Renewal"
        df.loc[mask, 'commission_rate'] = 0.014

        mask = (
            (df['ticket_type'] == "Full") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_department'] == "Premium") & (df['rep_sales_service'] == "Service") & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus Renewal"
        df.loc[mask, 'commission_rate'] = 0.01

        mask = (
            (df['ticket_type'] == "Premier") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_department'] == "Premium") & (df['rep_sales_service'] == "Service") & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Renewal"
        df.loc[mask, 'commission_rate'] = 0.01

        mask = (
            (df['ticket_type'] == "Full") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_sales_service'] == "Sales") & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus Renewal"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = (
            (df['ticket_type'] == "Premier") & (df['is_renewal'] == True) & (df['location_general'] != "Lounge 954") &
            (df['rep_sales_service'] == "Sales") & (df['location_general'] != "Suites")
        )
        df.loc[mask, 'commission_type'] = "Bowl Premier Renewal"
        df.loc[mask, 'commission_rate'] = 0.1

        mask = ((df['ticket_type'] == "Full") & (df['is_renewal'] == True) & (df['location_general'] == "Lounge 954"))
        df.loc[mask, 'commission_type'] = "Lounge 954 Premier Plus Renewal"
        df.loc[mask, 'commission_rate'] = 0.01

        mask = ((df['ticket_type'] == "Premier") & (df['is_renewal'] == True) & (df['location_general'] == "Lounge 954"))
        df.loc[mask, 'commission_type'] = "Lounge 954 Premier Renewal"
        df.loc[mask, 'commission_rate'] = 0.01


        ## Upgrade ##

        mask = ((df['ticket_type'] == "Full") & (df['is_upgrade'] == True) & (df['rep_sales_service'] == "Service"))
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus Upgrade"
        df.loc[mask, 'commission_rate'] = 0.05

        mask = ((df['ticket_type'] == "Full") & (df['is_upgrade'] == True) & (df['rep_department'].isin(["New Business", "Groups"])))
        df.loc[mask, 'commission_type'] = "Bowl Premier Plus Upgrade"
        df.loc[mask, 'commission_rate'] = 0.1

        ## Groups ##

        mask = ((df['ticket_type'] == "Groups") & (df['tier'].isin(["D", "E"])))
        df.loc[mask, 'commission_type'] = "Hockey D, E Tier Group"
        df.loc[mask,'commission_rate'] = 0.10

        mask = ((df['ticket_type'] == "Groups") & (df['tier'].isin(["B", "C"])))
        df.loc[mask, 'commission_type'] = "Hockey B, C Tier Group"
        df.loc[mask,'commission_rate'] = 0.06

        mask = ((df['ticket_type'] == "Groups") & (df['tier'].isin(["B", "C"])) & (df['rep_department'] == "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey B, C Tier Group - Premium"
        df.loc[mask,'commission_rate'] = 0.04

        mask = ((df['ticket_type'] == "Groups") & (df['tier'] == "A"))
        df.loc[mask, 'commission_type'] = "Hockey A Tier Group"
        df.loc[mask,'commission_rate'] = 0.04

        mask = ((df['ticket_type'] == "Groups") & (df['pc_one'].isin(["1", "2", "X", "Z"])) & (df['rep_department'] == "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey Premium Group"
        df.loc[mask,'commission_rate'] = 0.1

        ## Flex ##

        mask = ((df['ticket_type'] == "Flex") & (df['location_general'] != "Suites"))
        df.loc[mask, 'commission_type'] = "Flex"
        df.loc[mask,'commission_rate'] = 0.04

        ## Nightly Suites ##

        mask = ((df['ticket_type'] == "Nightly Suites") & (df['rep_department'] == "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey Nightly Suites - Premium"
        df.loc[mask,'commission_rate'] = 0.1

        mask = ((df['ticket_type'] == "Nightly Suites") & (df['rep_department'] != "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey Nightly Suites"
        df.loc[mask,'commission_rate'] = 0.05

        ## Stable ##

        mask = ((df['price_code'] == "YI") & (df['rep_department'] == "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey Nightly Stable - Premium"
        df.loc[mask,'commission_rate'] = 0.1

        mask = ((df['price_code'] == "YI") & (df['rep_department'] != "Premium"))
        df.loc[mask, 'commission_type'] = "Hockey Nightly Stable"
        df.loc[mask,'commission_rate'] = 0.05

        ## Double Commmission - Groups ##
        '''
            Group sales
                - add_date between 10/3 - 10/14
                - event_date in (10/6, 10/19, 10/23)
        '''

        mask = ((df['ticket_type'] == "Groups") & (df['add_datetime'] >= datetime(2022, 10, 3)) 
                & (df['add_datetime'] <= datetime(2022, 10, 14)) & (df['event_name'].isin(["22H1006E", "22H1019C", "22H1023D"])))
        df.loc[mask,'commission_rate'] = df.loc[mask,'commission_rate'] * 2

        return df

    @staticmethod
    def create_commissionable_amount(df: pd.DataFrame) -> pd.DataFrame:

        # df['commissionable_amount'] = df.apply(
        #     lambda row: (((row['block_purchase_price'] - row['food_fee']) / 1.07) * row['percent_paid']), axis = 1
        # )

        # df['commissionable_amount'] = df.apply(
        #     lambda row: (((row['gross_without_food']) / 1.07) * row['percent_paid']), axis = 1
        # )

        df['commissionable_amount'] = (df['gross_without_food'] / 1.07) * df['percent_paid']

        return df

    @staticmethod
    def calculate_total_commissions(df: pd.DataFrame) -> pd.DataFrame:

        # df['rep_commissions'] = df.apply(lambda row: (row['commissionable_amount'] * row['commission_rate']), axis = 1)

        df['rep_commissions'] = df['commissionable_amount'] * df['commission_rate']

        return df 

    @staticmethod
    def create_total_tax_field(df: pd.DataFrame) -> pd.DataFrame:

        df['total_tax'] = (df['net_revenue'] * 1.07) - df['net_revenue']

        return df
